In [1]:
!pip install pandas nltk tqdm

In [5]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/oushilin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/oushilin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oushilin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oushilin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/oushilin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
import pandas as pd
import re
import nltk
import ssl
from pathlib import Path
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tqdm import tqdm

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

desktop_path = Path.home() / "Desktop"
data_folder = desktop_path / "SOSC314" / "Data"
input_file = data_folder / "energy_narrative_W.csv"
output_file = data_folder / "energy_narrative_W_cleaned.csv"

def clean_text(text):
    if not isinstance(text, str):
        return ""
    
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    
    tokens = word_tokenize(text)
    
    stop_words = set(stopwords.words('english'))
    custom_stops = {
         'said', 'also', 'would', 'could', 'new', 'one', 
        'reuters', 'guardian', 'year', 'last', 'first', 'reported', 
        'told', 'mr', 'ms', 'says','tab','open','linkedin','facebook','state','email','see','image','reporting','link','OPEN','including'
    }
    stop_words.update(custom_stops)
    
    lemmatizer = WordNetLemmatizer()
    cleaned = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words and len(w) > 2]
    
    return " ".join(cleaned)

print(f"Loading raw Western media data...")
df = pd.read_csv(input_file)

print(f"Processing {len(df)} articles...")
tqdm.pandas()
df['cleaned_content'] = df['full_content'].progress_apply(clean_text)

df['publish_date'] = pd.to_datetime(df['publish_date'], errors='coerce')
df['year'] = df['publish_date'].dt.year
df['month'] = df['publish_date'].dt.month
df['day'] = df['publish_date'].dt.day

df = df.dropna(subset=['cleaned_content', 'publish_date'])
df = df[df['cleaned_content'] != ""]

df.to_csv(output_file, index=False, encoding='utf-8-sig')

print("\n" + "="*40)
print("Western Media Data Cleaning Complete")
print("-" * 40)
print(f"File saved: {output_file}")
print(f"Final article count: {len(df)}")
print("\nArticles by Year:")
print(df.groupby('year').size().to_string())
print("="*40)

[nltk_data] Downloading package punkt to /Users/oushilin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oushilin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oushilin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/oushilin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Loading raw Western media data...
Processing 2117 articles...


100%|██████████| 2117/2117 [00:04<00:00, 441.71it/s]



Western Media Data Cleaning Complete
----------------------------------------
File saved: /Users/oushilin/Desktop/SOSC314/Data/energy_narrative_W_cleaned.csv
Final article count: 2117

Articles by Year:
year
2020    348
2021    522
2022    525
2023    277
2024    234
2025    206
2026      5
